<a href="https://colab.research.google.com/github/Javeriatabraiz/Javeriatabraiz/blob/main/Copy_of_attentionresunet.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import os
import numpy as np
import pandas as pd
import random
import matplotlib.pyplot as plt
import cv2
from glob import glob
from sklearn.model_selection import train_test_split
from tqdm import tqdm_notebook as tqdm
from plotly.offline import iplot
import plotly as py
import plotly.tools as tls
import cufflinks as cf
from IPython.core.display import display, HTML
from pathlib import Path

import tensorflow as tf
from tensorflow.keras.utils import CustomObjectScope
from tqdm import tqdm
import keras
import imgaug.augmenters as iaa
from tensorflow.keras import models, layers, regularizers
from tensorflow.keras import backend as K

In [10]:
def conv_block(x, filter_size, size, dropout, batch_norm=False):
    
    conv = layers.Conv2D(size, (filter_size, filter_size), padding="same")(x)
    if batch_norm is True:
        conv = layers.BatchNormalization(axis=3)(conv)
    conv = layers.Activation("relu")(conv)

    conv = layers.Conv2D(size, (filter_size, filter_size), padding="same")(conv)
    if batch_norm is True:
        conv = layers.BatchNormalization(axis=3)(conv)
    conv = layers.Activation("relu")(conv)
    
    if dropout > 0:
        conv = layers.Dropout(dropout)(conv)

    return conv


def repeat_elem(tensor, rep):

     return layers.Lambda(lambda x, repnum: K.repeat_elements(x, repnum, axis=3),
                          arguments={'repnum': rep})(tensor)


def res_conv_block(x, filter_size, size, dropout, batch_norm=False):
    

    conv = layers.Conv2D(size, (filter_size, filter_size), padding='same')(x)
    if batch_norm is True:
        conv = layers.BatchNormalization(axis=3)(conv)
    conv = layers.Activation('relu')(conv)
    
    conv = layers.Conv2D(size, (filter_size, filter_size), padding='same')(conv)
    if batch_norm is True:
        conv = layers.BatchNormalization(axis=3)(conv)
    #conv = layers.Activation('relu')(conv)    #Activation before addition with shortcut
    if dropout > 0:
        conv = layers.Dropout(dropout)(conv)

    shortcut = layers.Conv2D(size, kernel_size=(1, 1), padding='same')(x)
    if batch_norm is True:
        shortcut = layers.BatchNormalization(axis=3)(shortcut)

    res_path = layers.add([shortcut, conv])
    res_path = layers.Activation('relu')(res_path)    #Activation after addition with shortcut (Original residual block)
    return res_path

def gating_signal(input, out_size, batch_norm=False):
    
    x = layers.Conv2D(out_size, (1, 1), padding='same')(input)
    if batch_norm:
        x = layers.BatchNormalization()(x)
    x = layers.Activation('relu')(x)
    return x

def attention_block(x, gating, inter_shape):
    shape_x = K.int_shape(x)
    shape_g = K.int_shape(gating)

# Getting the x signal to the same shape as the gating signal
    theta_x = layers.Conv2D(inter_shape, (2, 2), strides=(2, 2), padding='same')(x)  # 16
    shape_theta_x = K.int_shape(theta_x)

# Getting the gating signal to the same number of filters as the inter_shape
    phi_g = layers.Conv2D(inter_shape, (1, 1), padding='same')(gating)
    upsample_g = layers.Conv2DTranspose(inter_shape, (3, 3),
                                 strides=(shape_theta_x[1] // shape_g[1], shape_theta_x[2] // shape_g[2]),
                                 padding='same')(phi_g)  # 16

    concat_xg = layers.add([upsample_g, theta_x])
    act_xg = layers.Activation('relu')(concat_xg)
    psi = layers.Conv2D(1, (1, 1), padding='same')(act_xg)
    sigmoid_xg = layers.Activation('relu')(psi)
    shape_sigmoid = K.int_shape(sigmoid_xg)
    upsample_psi = layers.UpSampling2D(size=(shape_x[1] // shape_sigmoid[1], shape_x[2] // shape_sigmoid[2]))(sigmoid_xg)  # 32

    upsample_psi = repeat_elem(upsample_psi, shape_x[3])

    y = layers.multiply([upsample_psi, x])

    result = layers.Conv2D(shape_x[3], (1, 1), padding='same')(y)
    result_bn = layers.BatchNormalization()(result)
    return result_bn

In [11]:
def Attention_ResUNet(input_shape, NUM_CLASSES=1, dropout_rate=0.0, batch_norm=True):
    '''
    attention  baed Rsidual UNet
    
    '''
    # network structure
    FILTER_NUM = 64 # number of basic filters for the first layer
    FILTER_SIZE = 3 # size of the convolutional filter
    UP_SAMP_SIZE = 2 # size of upsampling filters
    # input data
    # dimension of the image depth
    inputs = layers.Input(input_shape, dtype=tf.float32)
    axis = 3

    # Downsampling layers
    # DownRes 1, double residual convolution + pooling
    conv_128 = res_conv_block(inputs, FILTER_SIZE, FILTER_NUM, dropout_rate, batch_norm)
    pool_64 = layers.MaxPooling2D(pool_size=(2,2))(conv_128)
    # DownRes 2
    conv_64 = res_conv_block(pool_64, FILTER_SIZE, 2*FILTER_NUM, dropout_rate, batch_norm)
    pool_32 = layers.MaxPooling2D(pool_size=(2,2))(conv_64)
    # DownRes 3
    conv_32 = res_conv_block(pool_32, FILTER_SIZE, 4*FILTER_NUM, dropout_rate, batch_norm)
    pool_16 = layers.MaxPooling2D(pool_size=(2,2))(conv_32)
    # DownRes 4
    conv_16 = res_conv_block(pool_16, FILTER_SIZE, 8*FILTER_NUM, dropout_rate, batch_norm)
    pool_8 = layers.MaxPooling2D(pool_size=(2,2))(conv_16)
    # DownRes 5, convolution only
    conv_8 = res_conv_block(pool_8, FILTER_SIZE, 16*FILTER_NUM, dropout_rate, batch_norm)

    # Upsampling layers
    # UpRes 6, attention gated concatenation + upsampling + double residual convolution
    gating_16 = gating_signal(conv_8, 8*FILTER_NUM, batch_norm)
    att_16 = attention_block(conv_16, gating_16, 8*FILTER_NUM)
    up_16 = layers.UpSampling2D(size=(UP_SAMP_SIZE, UP_SAMP_SIZE), data_format="channels_last")(conv_8)
    up_16 = layers.concatenate([up_16, att_16], axis=axis)
    up_conv_16 = res_conv_block(up_16, FILTER_SIZE, 8*FILTER_NUM, dropout_rate, batch_norm)
    # UpRes 7
    gating_32 = gating_signal(up_conv_16, 4*FILTER_NUM, batch_norm)
    att_32 = attention_block(conv_32, gating_32, 4*FILTER_NUM)
    up_32 = layers.UpSampling2D(size=(UP_SAMP_SIZE, UP_SAMP_SIZE), data_format="channels_last")(up_conv_16)
    up_32 = layers.concatenate([up_32, att_32], axis=axis)
    up_conv_32 = res_conv_block(up_32, FILTER_SIZE, 4*FILTER_NUM, dropout_rate, batch_norm)
    # UpRes 8
    gating_64 = gating_signal(up_conv_32, 2*FILTER_NUM, batch_norm)
    att_64 = attention_block(conv_64, gating_64, 2*FILTER_NUM)
    up_64 = layers.UpSampling2D(size=(UP_SAMP_SIZE, UP_SAMP_SIZE), data_format="channels_last")(up_conv_32)
    up_64 = layers.concatenate([up_64, att_64], axis=axis)
    up_conv_64 = res_conv_block(up_64, FILTER_SIZE, 2*FILTER_NUM, dropout_rate, batch_norm)
    # UpRes 9
    gating_128 = gating_signal(up_conv_64, FILTER_NUM, batch_norm)
    att_128 = attention_block(conv_128, gating_128, FILTER_NUM)
    up_128 = layers.UpSampling2D(size=(UP_SAMP_SIZE, UP_SAMP_SIZE), data_format="channels_last")(up_conv_64)
    up_128 = layers.concatenate([up_128, att_128], axis=axis)
    up_conv_128 = res_conv_block(up_128, FILTER_SIZE, FILTER_NUM, dropout_rate, batch_norm)

    # 1*1 convolutional layers
    
    conv_final = layers.Conv2D(NUM_CLASSES, kernel_size=(1,1))(up_conv_128)
    conv_final = layers.BatchNormalization(axis=axis)(conv_final)
    conv_final = layers.Activation('relu')(conv_final)  #Change to softmax for multichannel

    # Model integration
    model = models.Model(inputs, conv_final, name="AttentionResUNet")
    return model

IMG_HEIGHT = 256
IMG_WIDTH  = 256
IMG_CHANNELS = 3
num_labels = 1
input_shape = (IMG_HEIGHT,IMG_WIDTH,IMG_CHANNELS)


from tensorflow.keras import optimizers

att_res_unet_model = Attention_ResUNet(input_shape,dropout_rate=0.3)

opt =keras.optimizers.RMSprop(learning_rate=0.001)
metrics = ["acc"]

#att_res_unet_model.compile(loss='mse', optimizer=opt, metrics=metrics)
att_res_unet_model.compile(loss = 'mse',
              optimizer = opt,
              metrics = ["acc"])


# att_res_unet_model.compile(optimizer=Adam(lr = 1e-3), loss='binary_crossentropy', 
#               metrics=['accuracy', jacard_coef])

print(att_res_unet_model.summary())

size=256

img_data=[]

Model: "AttentionResUNet"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_2 (InputLayer)           [(None, 256, 256, 3  0           []                               
                                )]                                                                
                                                                                                  
 conv2d_48 (Conv2D)             (None, 256, 256, 64  1792        ['input_2[0][0]']                
                                )                                                                 
                                                                                                  
 batch_normalization_36 (BatchN  (None, 256, 256, 64  256        ['conv2d_48[0][0]']              
 ormalization)                  )                                                  

In [12]:
from tensorflow.keras.preprocessing.image import img_to_array

SIZE=256
img_data=[]

#fully sampked image --gt(y_train)
img=cv2.imread("/content/trainimage.jpg",1)
img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
img=cv2.resize(img,(SIZE, SIZE))

img_data.append(img_to_array(img))
img_array = np.reshape(img_data, (len(img_data), SIZE, SIZE, 3))
img_array = img_array.astype('float32') / 255
img_array = np.reshape(img_data, (len(img_data), SIZE, SIZE, 3))
y_train=img_array



#undersampled image XTRAIN
img_data1=[]
img1=cv2.imread("/content/undersampledimage.jpg",1)
img1=cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
img1=cv2.resize(img,(SIZE, SIZE))
img_data1.append(img_to_array(img1))
img_array1 = np.reshape(img_data1, (len(img_data1), SIZE, SIZE, 3))
img_array1 = img_array.astype('float32') / 255
img_array1 = np.reshape(img_data1, (len(img_data1), SIZE, SIZE, 3))
x_train=img_array1


#xtest image




In [ ]:
att_res_unet_history = att_res_unet_model.fit(y_train, x_train, epochs=20) #xtrain,xtest
x__train = np.stack(x_train, axis=0)  
plt.figure(figsize=(15,5))
loss = att_res_unet_history.history['loss']

epochs = range(1, len(loss) + 1)
plt.plot(epochs, loss, 'y', label='Training loss')

plt.title('Training and validation loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()
plt.show()

Epoch 1/20
1/1 [==============================] - 24s 24s/step - loss: 2884.0095 - acc: 0.1079
Epoch 2/20
1/1 [==============================] - 11s 11s/step - loss: 2851.9570 - acc: 0.0197
Epoch 3/20
1/1 [==============================] - 10s 10s/step - loss: 2826.5312 - acc: 0.0195
Epoch 4/20
1/1 [==============================] - 10s 10s/step - loss: 2819.9436 - acc: 0.0196
Epoch 5/20
1/1 [==============================] - 12s 12s/step - loss: 2819.8657 - acc: 0.0191
Epoch 6/20
1/1 [==============================] - 10s 10s/step - loss: 2814.1074 - acc: 0.0191
Epoch 7/20


In [ ]:
#xtest for prediction
img_data2=[]

img2=cv2.imread("/content/validationimage.jpg",1)
img2 = cv2.cvtColor(img2, cv2.COLOR_BGR2RGB)
img2=cv2.resize(img2,(SIZE, SIZE))

img_data2.append(img_to_array(img2))
img_array2 = np.reshape(img_data2, (len(img_data2), SIZE, SIZE, 3))
img_array2 = img_array.astype('float32') / 255
img_array2 = np.reshape(img_data2, (len(img_data2), SIZE, SIZE, 3))
x_test=img_array2

y=att_res_unet_model.predict(x_test)


In [ ]:
import matplotlib.pyplot as plt

plt.subplot(2,3,1)
plt.title('test_image')
plt.imshow(np.reshape(y_train,(256,256,3)),cmap='gray')


plt.subplot(2,3,2)
plt.title('gt image')
plt.imshow(np.reshape(x_train,(256,256,3)),cmap='gray')

plt.subplot(2,3,3)
plt.title('predicted')
plt.imshow(np.reshape(x_test,(256,256,3)),cmap='gray')

In [ ]:
import matplotlib.pyplot as plt

plt.subplot(2,3,1)
plt.title('test_image',)
plt.imshow(np.reshape(y_train,(256,256,3)),cmap='gray')


plt.subplot(2,3,2)
plt.title('gt image')
plt.imshow(np.reshape(x_train,(256,256,3)),cmap='gray')

plt.subplot(2,3,3)
plt.title('predicted')
plt.imshow(np.reshape(x_test,(256,256,3)),cmap='gray')